由于函数也是一个对象，而且函数对象可以被赋值给变量，所以，通过变量也能调用该函数。

In [2]:
def now():
    print('2015-3-25')
f = now
f()

2015-3-25


函数对象有一个__name__属性，可以拿到函数的名字：

In [5]:
print(now.__name__)
print(f.__name__)

now
now


如果想要增强now()函数的功能，比如在函数调用前自动打印日志，但又不希望修改now()函数的定义，这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。

本质上，decorator就是一个返回函数的高阶函数。所以，我们要定义一个能打印日志的decorator，可以定义如下：

In [109]:
def now():
    print('2015-3-25')

now = log(now)
def log(func):
    def wrapper(*args,**kw):
        print('call %s():'% func.__name__)# 执行__name__语句
        return func(*args,**kw)# 执行原函数now，返回结果
    return wrapper

now()

call now():
2015-3-25


In [151]:
@log
def now():
    print('2015-3-25')

def log(func):
    def wrapper(*args,**kw):
        print('call %s():'% func.__name__)
        return func(*args,**kw)
    return wrapper

now()

call now():
2015-3-25


观察上面的log，因为它是一个decorator，所以接受一个函数作为参数，并返回一个函数。我们要借助Python的@语法，把decorator置于函数的定义处：

In [14]:
@log
def now():
    print('2018-9-26')

调用now()函数，不仅会运行now()函数本身，还会在运行now()函数前打印一行日志：

In [8]:
now()

call now():
2018-9-26


把@log放到now()函数的定义处，相当于执行了语句：

In [ ]:
now = log(now)

由于log()是一个decorator，返回一个函数，所以，原来的now()函数仍然存在，只是现在同名的now变量指向了新的函数，于是调用now()将执行新函数，即在log()函数中返回的wrapper()函数。

wrapper()函数的参数定义是(*args, \**kw)，因此，wrapper()函数可以接受任意参数的调用。在wrapper()函数内，首先打印日志，再紧接着调用原始函数。

如果decorator本身需要传入参数，那就需要编写一个返回decorator的高阶函数，写出来会更复杂。比如，要自定义log的文本：

In [182]:
import functools

def log(text):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args,**kw):
            print('%s %s():'%(text,func.__name__))
            return func(*args,**kw)
        return wrapper
    return decorator

这个3层嵌套的decorator用法如下：

In [193]:
@log('execute')
def now():
    print('2018-9-26')
now()

execute now():
2018-9-26


和两层嵌套的decorator相比，3层嵌套的效果是这样的：

In [190]:
n=log('execute')
now=n(now)
now()

execute now():
execute now():
2018-9-26


In [191]:
now=log('execute')(now)
now()

execute now():
execute now():
execute now():
2018-9-26


我们来剖析上面的语句，首先执行log('execute')，返回的是decorator函数，再调用返回的函数，参数是now函数，返回值最终是wrapper函数。

## 练习
请设计一个decorator，它可作用于任何函数上，并打印该函数的执行时间：

In [215]:
import time, functools
def metric(fn):
    def decorator(*args,**kw):
        
        print('%s executed in %s ms' % (fn.__name__, 10.24))
        return fn(*args,**kw)
    return decorator

@metric
def fast(x, y):
    time.sleep(0.0012)
    return x + y;

fast(5,8)

fast executed in 10.24 ms


NameError: name 'x' is not defined

很多人对装饰器难以理解，原因是由于以下三点内容没有搞清楚：
- 关于函数“变量”（或“变量”函数）的理解
- 关于高阶函数的理解
- 关于嵌套函数的理解

## 1、装饰器
装饰器实际上就是为了给某程序增添功能，但该程序已经上线或已经被使用，那么就不能大批量的修改源代码，这样是不科学的也是不现实的，因为就产生了装饰器，使得其满足：
- 不能修改被装饰的函数的源代码
- 不能修改被装饰的函数的调用方式
- 满足1、2的情况下给程序增添功能

那么根据需求，同时满足了这三点原则，这才是我们的目的。因为，下面我们从解决这三点原则入手来理解装饰器。

#### 装饰器的原则组成：
**< 函数+实参高阶函数+返回值高阶函数+嵌套函数+语法糖 = 装饰器 >**
这个式子是贯穿装饰器的灵魂所在！

### 2、需求的实现

In [18]:
import time
def test():
    time.sleep(2)
    print("test is running!")
test()

test is running!


那么要求在满足三原则的基础上，给程序添加统计运行时间（2 second）功能。

在行动之前，我们先来看一下文章开头提到的原因1（关于函数“变量”（或“变量”函数）的理解）。

In [248]:
import time

def test():
    time.sleep(2)
    print("test is running!")
    
def cal(func):
    start=time.time()
    func()
    end=time.time()
    return end-start
cal(test)

<function test at 0x0000024544095C80>
test is running!


2.000596523284912

这修改了调用方式,不修改调用方式，那么在这样的程序中，被装饰函数就无法传递到另一个装饰函数中去。

如果不修改调用方式，就是一定要有test()这条语句，那么就用到了第二种高阶函数，即**返回值中包含函数名**

In [247]:
import time

def test():
    time.sleep(2)
    print("test is running!")
@cal
def cal(func):
    print(func)
    return func

test()

<function cal at 0x0000024543FD6048>
test is running!


我们看这段代码，在#3处，将test传入deco()，在deco()里面操作之后，最后返回了func，并赋值给t。因此这里test => func => t，都是一样的函数体。最后在#4处保留了原来的函数调用方式。 

看到这里显然会有些困惑，我们的需求不是要计算函数的运行时间么，怎么改成输出函数地址了。是因为，单独采用第二种高阶函数（返回值中包含函数名）的方式，并且保留原函数调用方式，是无法计时的。如果在deco()里计时，显然会执行一次，而外面已经调用了test()，会重复执行。这里只是为了说明第二种高阶函数的思想，下面才真的进入重头戏。

In [249]:
import time

def test():
    time.sleep(2)
    print("test is running!")
    
def timer(func):    
    def cal():
        start=time.time()
        func()
        end=time.time()
        return end-start
    return cal
cal(test)

test is running!


2.0008761882781982

这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性：

In [61]:
class Student(object):

    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value

    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self,value):
        self._age=self._birth-value
    


In [63]:
s = Student()

s.birth=2003
s.age=25
s.age

1978

## 练习
请利用@property给一个Screen对象加上width和height属性，以及一个只读属性resolution：

In [257]:
class Screen(object):
    
    @property
    def size(self):
        return self.width,self.height
    
    @size.setter
    def size(self,value):
        self.width,self.height=value
        
    @property
    def resolution(self):
        return self.width*self.height

s=Screen()
s.size=1024,768
s.size
s.width
s.height
s.resolution

786432

In [264]:
# 装饰无参函数
import time

# 装饰器主体，func为‘待升级函数’，作用为获得功能函数名deco
def timer(func):
    # 需求功能函数，封装起来
    def deco(*args,**kw):  
        start = time.time()
        func(*args,**kw)
        stop = time.time()
        print(stop-start)
    # 将功能函数名返回，等待被赋值
    return deco

'''
test = timer(test) 
timer(test)得到的为deco，将这个函数名赋予给了test
等价于：
@timer
'''
@timer
def test(pap):
    time.sleep(2)
    print("test is running!",pap)   
test(3) 

test is running! 3
2.0000972747802734


# 总结
装饰器相当于是将一个‘需求功能函数’封装起来，然后将它的函数名赋予给‘待升级功能函数名’，运行起来就是，原函数、新功能函数各运行一遍。日后继续深入研究。

# 偏函数
Python的functools模块提供了很多有用的功能，其中一个就是偏函数（Partial function）。要注意，这里的偏函数和数学意义上的偏函数不一样。

偏函数函数式为`unctools.partial`,直接创建一个偏函数，不需要我们自己定义int2()，可以直接使用下面的代码创建一个新的函数int2：

In [267]:
import functools
int2 = functools.partial(int, base=2)
print(int2('1000000'))
print(int2('1010101'))


64
85


简单总结functools.partial的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。

注意到上面的新的int2函数，仅仅是把base参数重新设定默认值为2，但也可以在函数调用时传入其他值：

In [268]:
int2('1000000', base=10)

1000000

6


最后，创建偏函数时，实际上可以接收函数对象、*args和\**kw这3个参数，当传入：

In [29]:
int2 = functools.partial(int, base=2)

实际上固定了int()函数的关键字参数base，也就是：

In [26]:
int2('10010')
# 相当于：
kw = { 'base': 2 }
int('10010', **kw)

18

再举一个例子：

In [270]:
import functools
max2 = functools.partial(max)
max2(1,4,6)

6

In [276]:
max2 = functools.partial(max, 10)
max2(1,4,6)

10

实际上会把10作为*args的一部分自动加到左边，也就是：

In [32]:
max2(5, 6, 7)
# 相当于：
args = (10, 5, 6, 7)
max(*args)

10

当函数的参数个数太多，需要简化时，使用functools.partial可以创建一个新的函数，这个新函数可以固定住原函数的部分参数，从而在调用时更简单。